In [1]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json_shot_scraper import flatten_shot, flatten_goal, flatten_complete_pass, flatten_incomplete_pass, flatten_corner
from  player_scraper import flatten_player, flatten_sub
from dataframe_cleaner import (pass_to_shot, corner_to_shot, transpose_coordinates, coord_to_yards, 
                               shot_distance_angle, dummy_columns, drop_own_goals, goal_dummy)

In [25]:
pd.set_option('display.max_columns', 50)

In [2]:
from html_scraper import db

In [27]:
from mongo_to_db import game_to_cleaned_df

In [3]:
db.list_collection_names()

['games']

In [5]:
db.games.count()

/Users/david/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


166

In [7]:
game = db.games.find_one()

In [8]:
pprint.pprint(game)

{'Revision': '$Revision: 1318 $',
 '_id': ObjectId('5bfb4d8a3d54c03054b19f7f'),
 'broadcast': {'media': [{'mediaId': 10059, 'name': 'Fox Sports Premium'}]},
 'commentary': {'22336291': {'t': 'status', 'ts': 1533938496},
                '22336293': {'t': 'clearances', 'ts': 1533938527},
                '22336296': {'t': 'cornerKicks', 'ts': 1533938544},
                '22336297': {'t': 'goals', 'ts': 1533938558},
                '22336314': {'t': 'fouls', 'ts': 1533938628},
                '22336318': {'t': 'clearances', 'ts': 1533938694},
                '22336327': {'t': 'throwIn', 'ts': 1533938722},
                '22336329': {'t': 'clearances', 'ts': 1533938726},
                '22336338': {'t': 'cornerKicks', 'ts': 1533938756},
                '22336342': {'t': 'goalkick', 'ts': 1533938783},
                '22336351': {'t': 'clearances', 'ts': 1533938814},
                '22336355': {'t': 'fouls', 'ts': 1533938824},
                '22336357': {'t': 'yellowCards', 'ts': 153393

                                               'team': 13,
                                               'type': 180},
                                  '22336310': {'coord': {'1': {'x': 0.45,
                                                               'y': 0.46,
                                                               'z': -1},
                                                         '2': {'x': 0.75,
                                                               'y': 0.13,
                                                               'z': -1}},
                                               'plyrId': 10601,
                                               'recvId': 15325,
                                               't': {'half': 1,
                                                     'm': 1,
                                                     's': 59},
                                               'team': 13,
                                               'type': 180},
 

                                  '22336341': {'coord': {'1': {'x': 0.78,
                                                               'y': 0.33,
                                                               'z': -1},
                                                         '2': {'x': 0.78,
                                                               'y': 0.33,
                                                               'z': -1}},
                                               'plyrId': 172756,
                                               'recvId': 42505,
                                               't': {'half': 1,
                                                     'm': 3,
                                                     's': 19},
                                               'team': 20,
                                               'type': 180},
                                  '22336344': {'coord': {'1': {'x': 0.78,
                                              

                                               'type': 180},
                                  '22336415': {'coord': {'1': {'x': -0.27,
                                                               'y': -0.77,
                                                               'z': -1},
                                                         '2': {'x': 0.08,
                                                               'y': -0.48,
                                                               'z': -1}},
                                               'plyrId': 69857,
                                               'recvId': 70151,
                                               't': {'half': 1,
                                                     'm': 11,
                                                     's': 56},
                                               'team': 13,
                                               'type': 180},
                                  '22336417': {'coord': 

                                                               'z': -1},
                                                         '2': {'x': 0.46,
                                                               'y': 0.33,
                                                               'z': -1}},
                                               'plyrId': 204820,
                                               'recvId': 33777,
                                               't': {'half': 1,
                                                     'm': 19,
                                                     's': 41},
                                               'team': 13,
                                               'type': 180},
                                  '22336510': {'coord': {'1': {'x': 0.55,
                                                               'y': 0.15,
                                                               'z': -1},
                                              

...

                                                                 'z': -1}},
                                                 'plyrId': 172756,
                                                 't': {'half': 2,
                                                       'm': 68,
                                                       's': 10},
                                                 'team': 20,
                                                 'type': 181},
                                    '22337101': {'coord': {'1': {'x': -0.19,
                                                                 'y': 0.77,
                                                                 'z': -1},
                                                           '2': {'x': -0.19,
                                                                 'y': 0.77,
                                                                 'z': -1}},
                                                 'plyrId': 19132,
               

                                                       'y': -0.02,
                                                       'z': 0.6}},
                                       'ctchBy': 15325,
                                       'plyrId': 94225,
                                       't': {'half': 2, 'm': 59, 's': 31},
                                       'team': 20,
                                       'type': 35},
                          '22337018': {'coord': {'1': {'x': -0.8,
                                                       'y': 0.1,
                                                       'z': -1},
                                                 '2': {'x': -0.99,
                                                       'y': -0.06,
                                                       'z': -1}},
                                       'plyrId': 185745,
                                       't': {'half': 2, 'm': 61, 's': 17},
                                       'team': 2

                                                     'm': 91,
                                                     's': 27},
                                               'team': 20,
                                               'type': 7}},
                'throwIn': {'22336327': {'coord': {'1': {'x': 0.48,
                                                         'y': -0.98,
                                                         'z': -1},
                                                   '2': {'x': 0.75,
                                                         'y': -0.81,
                                                         'z': -1}},
                                         'plyrId': 208534,
                                         't': {'half': 1, 'm': 3, 's': 46},
                                         'team': 20,
                                         'type': 46},
                            '22336359': {'coord': {'1': {'x': -0.06,
                                    

                            '22337271': {'coord': {'1': {'x': 0.47,
                                                         'y': -0.98,
                                                         'z': -1},
                                                   '2': {'x': 0.56,
                                                         'y': -0.54,
                                                         'z': -1}},
                                         'plyrId': 204820,
                                         't': {'half': 2, 'm': 88, 's': 50},
                                         'team': 13,
                                         'type': 46},
                            '22337273': {'coord': {'1': {'x': 0.48,
                                                         'y': -0.99,
                                                         'z': -1},
                                                   '2': {'x': 0.15,
                                                         'y': -0.72,
       

                 '20': {'homeAggregateScore': '',
                        'score': 2,
                        'scoreDisciplinary': None,
                        'scorePenaltyRow': None,
                        'scorePenaltyRowD': None}},
 'status': {'compactValue': 'Fin',
            'currentMinutes': 96,
            'currentPeriod': 2,
            'extraTime': False,
            'lastPeriodMinutes': 49,
            'lastPeriodSeconds': 7,
            'lineUpConfirmed': True,
            'shootout': False,
            'startHour': None,
            'startMinute': None,
            'startSecond': None,
            'statusId': 2,
            'value': 'Finalizado'},
 'summary': {'allPasses': {'awayQty': 283,
                           'homeQty': 275,
                           'perPlayerQty': {'10601': 44,
                                            '121341': 17,
                                            '124151': 5,
                                            '151431': 6,
             

In [14]:
shots = list(game['incidences']['shots'].items())
game_id = game['match']['matchId']
shot_list_dicts = [flatten_shot(shot, game_id) for shot in shots]
shot_df = pd.DataFrame(shot_list_dicts)
shot_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min)
0,448548,99779.0,0.85,1.00,0.05,-0.04,-1,0.83,22336297,11,20,1.033333
1,448548,60730.0,0.52,0.73,0.14,-0.03,-1,0.73,22336386,35,20,9.683333
2,448548,177285.0,0.73,0.96,0.00,-0.10,-1,0.21,22336583,13,20,26.833333
3,448548,185745.0,0.81,0.98,0.48,-0.22,-1,-1.00,22336723,33,20,38.883333
4,448548,77892.0,-0.50,-0.99,-0.46,0.28,-1,-1.00,22336739,33,13,40.083333


In [15]:
completed_passes = list(game['incidences']['correctPasses'].items())
completed_list_dicts = [flatten_complete_pass(apass, game_id) for apass in completed_passes]
completed_passes_df = pd.DataFrame(completed_list_dicts)
completed_passes_df.head()

,game_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,pass_player,pass_type,rec_player,team_id,time_of_event(min)
0,448548,-0.02,-0.33,-0.04,-0.14,-1,-1,42505.0,180,60730.0,20,0.116667
1,448548,-0.61,-0.61,-0.63,-0.63,-1,-1,60730.0,180,208534.0,20,0.150000
2,448548,-0.56,-0.56,-0.25,-0.25,-1,-1,208534.0,180,94225.0,20,0.200000
3,448548,-0.52,-0.52,0.79,0.79,-1,-1,94225.0,180,30896.0,20,0.250000
4,448548,-0.48,-0.48,-0.05,-0.05,-1,-1,30896.0,180,99779.0,20,0.283333


In [16]:
shot_pass_df = pass_to_shot(shot_df, completed_passes_df)
shot_pass_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2
0,448548,99779.0,0.85,1.00,0.05,-0.04,-1,0.83,22336297,11,20,1.033333,193955.0,0.89,0.85,0.47,-0.19,-1.0,-1.0
1,448548,60730.0,0.52,0.73,0.14,-0.03,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,448548,177285.0,0.73,0.96,0.00,-0.10,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,448548,185745.0,0.81,0.98,0.48,-0.22,-1,-1.00,22336723,33,20,38.883333,177285.0,0.49,0.86,0.13,0.65,-1.0,-1.0
4,448548,77892.0,-0.50,-0.99,-0.46,0.28,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
corners = list(game['incidences']['cornerKicks'].items())
corner_dicts = [flatten_corner(kick, game_id) for kick in corners]
corner_df = pd.DataFrame(corner_dicts)
corner_df.head()

,ck_coord_x1,ck_coord_x2,ck_coord_y1,ck_coord_y2,ck_coord_z1,ck_coord_z2,ck_id,game_id,player_id,time_of_event(min)
0,0.98,0.72,0.97,-0.16,-1,-1,22336296,448548,177285.0,0.800000
1,0.96,0.81,-0.94,-0.29,-1,-1,22336338,448548,177285.0,4.333333
2,0.98,0.74,1.00,-0.02,-1,-1,22336570,448548,185745.0,24.750000
3,0.97,0.72,-0.95,-0.06,-1,-1,22336593,448548,185745.0,29.133333
4,0.99,0.73,0.99,-0.10,-1,-1,22336716,448548,185745.0,38.450000


In [19]:
shot_pass_corner = corner_to_shot(shot_pass_df, corner_df)
shot_pass_corner.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick
0,448548,99779.0,0.85,1.00,0.05,-0.04,-1,0.83,22336297,11,20,1.033333,177285.0,0.98,0.72,0.97,-0.16,-1.0,-1.0,1.0
1,448548,60730.0,0.52,0.73,0.14,-0.03,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,448548,177285.0,0.73,0.96,0.00,-0.10,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,448548,185745.0,0.81,0.98,0.48,-0.22,-1,-1.00,22336723,33,20,38.883333,177285.0,0.49,0.86,0.13,0.65,-1.0,-1.0,0.0
4,448548,77892.0,-0.50,-0.99,-0.46,0.28,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [20]:
transposed_df = transpose_coordinates(shot_pass_corner)
transposed_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick
0,448548,99779.0,0.85,1.00,0.05,-0.04,-1,0.83,22336297,11,20,1.033333,177285.0,0.98,0.72,0.97,-0.16,-1.0,-1.0,1.0
1,448548,60730.0,0.52,0.73,0.14,-0.03,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,448548,177285.0,0.73,0.96,0.00,-0.10,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,448548,185745.0,0.81,0.98,0.48,-0.22,-1,-1.00,22336723,33,20,38.883333,177285.0,0.49,0.86,0.13,0.65,-1.0,-1.0,0.0
4,448548,77892.0,0.50,0.99,0.46,-0.28,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [21]:
yard_df = coord_to_yards(transposed_df)
yard_df

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,448548,77892.0,11.56,0.44,10.59,-1.21,-1,0.14,22336802,35,13,44.483333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,448548,42505.0,11.11,1.33,11.50,4.24,-1,0.25,22336958,35,20,54.983333,177285.0,49.78,16.44,-11.50,19.67,-1.0,-1.0,0.0
7,448548,94225.0,12.44,1.78,-2.42,0.61,-1,0.60,22337005,35,20,59.516667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,448548,185745.0,8.89,0.44,-3.03,1.82,-1,-1.00,22337018,33,20,61.283333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,448548,36681.0,13.78,1.78,8.78,-3.03,-1,0.19,22337074,35,13,68.550000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [22]:
shot_distance_df = shot_distance_angle(yard_df)
shot_distance_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,...,passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,...,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,...,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607


In [23]:
df = dummy_columns(shot_distance_df)
df1 = drop_own_goals(df)
df2 = goal_dummy(df1)

In [26]:
df2

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0
5,448548,77892.0,11.56,0.44,10.59,-1.21,-1,0.14,22336802,35,13,44.483333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.677426,42.492487,0,0,0
6,448548,42505.0,11.11,1.33,11.50,4.24,-1,0.25,22336958,35,20,54.983333,177285.0,49.78,16.44,-11.50,19.67,-1.0,-1.0,0.0,15.990063,45.988197,1,0,0
7,448548,94225.0,12.44,1.78,-2.42,0.61,-1,0.60,22337005,35,20,59.516667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.673200,-11.008472,0,0,0
8,448548,185745.0,8.89,0.44,-3.03,1.82,-1,-1.00,22337018,33,20,61.283333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.392178,-18.820773,0,0,0
9,448548,36681.0,13.78,1.78,8.78,-3.03,-1,0.19,22337074,35,13,68.550000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.339425,32.503466,0,0,0


In [28]:
game_to_cleaned_df(game)

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0
5,448548,77892.0,11.56,0.44,10.59,-1.21,-1,0.14,22336802,35,13,44.483333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.677426,42.492487,0,0,0
6,448548,42505.0,11.11,1.33,11.50,4.24,-1,0.25,22336958,35,20,54.983333,177285.0,49.78,16.44,-11.50,19.67,-1.0,-1.0,0.0,15.990063,45.988197,1,0,0
7,448548,94225.0,12.44,1.78,-2.42,0.61,-1,0.60,22337005,35,20,59.516667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.673200,-11.008472,0,0,0
8,448548,185745.0,8.89,0.44,-3.03,1.82,-1,-1.00,22337018,33,20,61.283333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.392178,-18.820773,0,0,0
9,448548,36681.0,13.78,1.78,8.78,-3.03,-1,0.19,22337074,35,13,68.550000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.339425,32.503466,0,0,0
